In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/2019_nCoV_data']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv')

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove plotting
# import seaborn as sns
# import matplotlib.pyplot as plt
# import datetime

# plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
# sns.set_style("dark")
# sns.set()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
#Load the data and a quick view
def adjustformat(x):
    if '1/31/2020' in x:
        return '2020-31-01 ' + x.split()[1] + ":00"
    else:
        return x
    
corona_df = pd.read_csv("./input/2019_nCoV_data.scaled.csv")
corona_df1 = corona_df[:497]
corona_df1['Date'] = pd.to_datetime(corona_df1['Date'])
corona_df2 = corona_df[497:]
corona_df2['Date'] = corona_df2['Date'].apply(lambda x: adjustformat(x))
corona_df2['Date'] = pd.to_datetime(corona_df2['Date'],format = '%Y-%d-%m %H:%M:%S')
# corona_df2.head()
corona_df = corona_df1.append(corona_df2)
corona_df.head()

/tmp/ipykernel_379901/2240193437.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corona_df1['Date'] = pd.to_datetime(corona_df1['Date'])
/tmp/ipykernel_379901/2240193437.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corona_df2['Date'] = corona_df2['Date'].apply(lambda x: adjustformat(x))
/tmp/ipykernel_379901/2240193437.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22 12:00:00,Anhui,China,1/22/2020 12:00,1.0,0.0,0.0
1,2,2020-01-22 12:00:00,Beijing,China,1/22/2020 12:00,14.0,0.0,0.0
2,3,2020-01-22 12:00:00,Chongqing,China,1/22/2020 12:00,6.0,0.0,0.0
3,4,2020-01-22 12:00:00,Fujian,China,1/22/2020 12:00,1.0,0.0,0.0
4,5,2020-01-22 12:00:00,Gansu,China,1/22/2020 12:00,0.0,0.0,0.0


In [3]:
#convert the Last update column to datetime
corona_df['Date'] = pd.to_datetime(corona_df['Date'])

In [4]:
#Quick check on data distribution
corona_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Sno             631 non-null    int64         
 1   Date            631 non-null    datetime64[ns]
 2   Province/State  487 non-null    object        
 3   Country         631 non-null    object        
 4   Last Update     631 non-null    object        
 5   Confirmed       631 non-null    float64       
 6   Deaths          631 non-null    float64       
 7   Recovered       631 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 39.6+ KB


We dont have missing values in counts of Confirmed, Deaths or Recovered.

In [5]:
#Lets do one more check
corona_df[(corona_df.Confirmed == 0) & (corona_df.Deaths == 0) & (corona_df.Recovered ==0)]

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
4,5,2020-01-22 12:00:00,Gansu,China,1/22/2020 12:00,0.0,0.0,0.0
10,11,2020-01-22 12:00:00,Heilongjiang,China,1/22/2020 12:00,0.0,0.0,0.0
12,13,2020-01-22 12:00:00,Hong Kong,China,1/22/2020 12:00,0.0,0.0,0.0
15,16,2020-01-22 12:00:00,Inner Mongolia,China,1/22/2020 12:00,0.0,0.0,0.0
18,19,2020-01-22 12:00:00,Jilin,China,1/22/2020 12:00,0.0,0.0,0.0
22,23,2020-01-22 12:00:00,Qinghai,China,1/22/2020 12:00,0.0,0.0,0.0
23,24,2020-01-22 12:00:00,Shaanxi,China,1/22/2020 12:00,0.0,0.0,0.0
30,31,2020-01-22 12:00:00,Tibet,China,1/22/2020 12:00,0.0,0.0,0.0
32,33,2020-01-22 12:00:00,Xinjiang,China,1/22/2020 12:00,0.0,0.0,0.0
53,54,2020-01-23 12:00:00,Inner Mongolia,Mainland China,1/23/20 12:00 PM,0.0,0.0,0.0


> So, 19 rows with 0 values on all 3 columns. Lets keep it as this is just visualization task

In [6]:
#Lets add day count to look at the trend
start_date = corona_df.sort_values(['Date'], ascending=[True]).head(1).values[0][1]
print(start_date)
#start_date = corona_df['Last Update'].min()
corona_df['nthday'] = corona_df['Date'].apply(lambda x: ((x - start_date).days)+1)
current_day = corona_df['nthday'].max()

2020-01-22 12:00:00


In [7]:
#Checking how to take the counts.
corona_df['Confirmed'].sum()

78269.0

> Too many cases were found to be Confirmed, which is not true. So we're getting the data cumulatively for each day.

In [8]:
corona_df[corona_df.nthday == current_day]['Confirmed'].sum()

17295.0

In [9]:
#Lets quickly plot the trend.
# FIRST-AUTHOR: remove plotting
# corona_df.groupby(['nthday'])['Confirmed'].sum().plot()
corona_df.groupby(['nthday'])['Confirmed'].sum()

nthday
1       555.0
2       653.0
3       941.0
4      2019.0
5      2794.0
6      4473.0
7      6057.0
8      7783.0
9      9776.0
10    11374.0
11    14549.0
12    17295.0
Name: Confirmed, dtype: float64

The spread is exponential after 5th day which makes sense as people travel after chinese new year.

In [10]:
#death rate vs recovery rate
dr_rate = corona_df.groupby(['nthday'])['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()
dr_rate['Deathrate'] = (dr_rate['Deaths'] / dr_rate['Confirmed'])*100
dr_rate['Recoveryrate'] = (dr_rate['Recovered'] / dr_rate['Confirmed'])*100

# FIRST-AUTHOR: remove plotting
# sns.lineplot(x='nthday', y='Deathrate', data=dr_rate, label="Death", color='red')
# sns.lineplot(x='nthday', y='Recoveryrate', data=dr_rate, label="Recovery", color='green')

/tmp/ipykernel_379901/229556944.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dr_rate = corona_df.groupby(['nthday'])['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()


As we can see there is sudden decrease in the rate at which recovery happens after day 2. But the death rate keeping at almost constant rate till day 10. Also we can see recovery rate is increasing and wishing to go even higher.

In [11]:
#No of countries affected so far
corona_df[corona_df.nthday == current_day].groupby(['Country'])['Confirmed', 'Deaths','Recovered' ].sum().sort_values(['Confirmed'], ascending=[False])

/tmp/ipykernel_379901/2955947347.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  corona_df[corona_df.nthday == current_day].groupby(['Country'])['Confirmed', 'Deaths','Recovered' ].sum().sort_values(['Confirmed'], ascending=[False])


,Confirmed,Deaths,Recovered
Country,,,
Mainland China,17114.0,361.0,478.0
Japan,20.0,0.0,1.0
Thailand,19.0,0.0,5.0
Singapore,18.0,0.0,0.0
Hong Kong,15.0,0.0,0.0
South Korea,15.0,0.0,0.0
Australia,12.0,0.0,2.0
Taiwan,10.0,0.0,0.0
Germany,10.0,0.0,0.0


China is worst affected while Thailand is recovering fast. We can also see countries with most spread after China.

In [12]:
#Lets see the country spread by the day
corona_df.groupby(['Country']).agg({'nthday' : 'min'}).reset_index().sort_values(['nthday'])

,Country,nthday
28,US,1
26,Thailand,1
4,China,1
21,South Korea,1
12,Japan,1
0,Australia,2
25,Taiwan,2
20,Singapore,2
18,Philippines,2
16,Mexico,2


Interestingly, 'Mainland China' has the first case on 2nd day while US has first case on first day. Maybe missing data. Also India has the first case on 9th day.

In [13]:
#
# FIRST-AUTHOR: remove plotting
# corona_df.groupby(['nthday'])['Country'].nunique().plot()
corona_df.groupby(['nthday'])['Country'].nunique()

nthday
1      5
2     15
3     11
4     14
5     15
6     19
7     18
8     20
9     23
10    27
11    27
12    27
Name: Country, dtype: int64

In [14]:
corona_df[(corona_df.nthday == current_day) & (corona_df.Country.isin(['China', 'Mainland China']))].groupby('Province/State')['Confirmed', 'Deaths', 'Recovered'].sum().reset_index().sort_values(['Confirmed'], ascending=[False])

/tmp/ipykernel_379901/320904281.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  corona_df[(corona_df.nthday == current_day) & (corona_df.Country.isin(['China', 'Mainland China']))].groupby('Province/State')['Confirmed', 'Deaths', 'Recovered'].sum().reset_index().sort_values(['Confirmed'], ascending=[False])


,Province/State,Confirmed,Deaths,Recovered
12,Hubei,11177.0,350.0,295.0
30,Zhejiang,724.0,0.0,36.0
5,Guangdong,683.0,0.0,15.0
11,Henan,566.0,2.0,14.0
13,Hunan,521.0,0.0,16.0
0,Anhui,408.0,0.0,7.0
16,Jiangxi,391.0,0.0,18.0
2,Chongqing,300.0,2.0,7.0
15,Jiangsu,271.0,0.0,7.0
25,Sichuan,254.0,1.0,12.0


Hubei has been at the receiving end of Corona both at the spread and death. 

Hubei seems to be economically (GDP) doing good among other provinces which may attract other countries. Also Wuhan is most populous city in Hubei and one among in China

https://en.wikipedia.org/wiki/Hubei

https://www.chinahighlights.com/travelguide/top-large-cities.htm